In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [3]:
ahu = pd.read_csv("/content/drive/MyDrive/ime_project/data/porest_area_ahu_automatic.csv")
chiller = pd.read_csv("/content/drive/MyDrive/ime_project/data/porest_area_chiller_automatic.csv")

ahu = ahu[['ahu_no','building','power','inv','occupancy','out_temp','in_temp','zone_min_temp','cdate']]
chiller = chiller[['chiller_no','temp','power','inv','cdate']]

# AHU 전처리

In [4]:
# ahu 전처리, A 구역 없앤뒤
ahu_a = ahu[ahu['building'] == 'A']
ahu_a.drop(columns=['building'], inplace=True)
# 원래 ahu 칼럼에 있던 관측치를 기존제어라고 명명
ahu_a['how_0']='기존제어'

# 시계열 형태로 변경
ahu_a.set_index('cdate', inplace=True)
ahu_a.index = pd.to_datetime(ahu_a.index)

ahu_group = ahu_a.groupby('ahu_no')

start_date = '2023-01-31 10:00:00'
end_date = '2023-08-01 16:00:00'

ahu_dct = {}
for name, group_df in ahu_group:
  group_df.index = group_df.index.floor('10T') # 기존의 datetime을 10분 단위로 내림
  group_df = group_df.loc[~group_df.index.duplicated(keep='first')] # 내리는 과정에서 중복 발생시 중복 제거

  date_range = pd.date_range(start=start_date, end=end_date, freq='10T')
  group_df = group_df.reindex(date_range) # 10분 단위로 row를 생성

  ahu_dct[name] = group_df

<ipython-input-4-3225f3f5321b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ahu_a.drop(columns=['building'], inplace=True)
<ipython-input-4-3225f3f5321b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ahu_a['how_0']='기존제어'


In [5]:
ahu_dct[1].loc[:'2023-02-01 11'].head()

,ahu_no,power,inv,occupancy,out_temp,in_temp,zone_min_temp,how_0
2023-01-31 10:00:00,1.0,1.0,30.0,8.0,24.0,\N,\N,기존제어
2023-01-31 10:10:00,1.0,1.0,30.0,8.0,24.0,\N,\N,기존제어
2023-01-31 10:20:00,1.0,1.0,30.0,8.0,25.0,\N,\N,기존제어
2023-01-31 10:30:00,1.0,1.0,30.0,8.0,25.0,\N,\N,기존제어
2023-01-31 10:40:00,1.0,1.0,30.0,8.0,25.0,\N,\N,기존제어


In [6]:
alog = pd.read_csv("/content/drive/MyDrive/ime_project/data/porest_area_log.csv", usecols=[2, 3, 4, 6, 7],
                    names=['type', 'mode', 'text', 'cdate_0', 'cdate_1'], header=None)
alog['text'] = alog['text'].str.replace('::',',').str.replace('[','').str.replace(']',',').str.replace('\\','').str.replace(':',',')
alog = alog[(alog['type'] == 'freezer')|(alog['type'] == 'ahu')].reset_index(drop=True)
alog[['device', 'how_0', 'how_1', 'power']] = pd.DataFrame(alog.text.str.rsplit(',', 3).tolist())
alog[['name', 'day']] = pd.DataFrame(alog.device.str.rsplit(',', 1).tolist())
alog['device'] = alog['name'].str.replace(',','')
alog['cdate'] = np.where(pd.isna(alog['cdate_0']), alog['cdate_1'], alog['cdate_0'])
alog['cdate'] = pd.to_datetime(alog['cdate'])
alog['cdate'] = alog['cdate'].dt.round('10min')

alog_filtered = alog[['type', 'mode', 'device', 'how_0', 'how_1', 'power', 'day', 'cdate']]

# device를 name과 number로 분리
alog_filtered['dname'] = alog_filtered['device'].str.extract('([가-힣]+)')
alog_filtered['dnumber'] = alog_filtered['device'].str.extract('(\d+)')
alog_filtered['dnumber'] = alog_filtered['dnumber'].astype('int')
alog_filtered.drop('device', axis=1, inplace=True)
# 시계열 형태로 변경
alog_filtered.set_index('cdate', inplace=True)
alog_filtered.index = pd.to_datetime(alog_filtered.index)

alog_filtered = alog_filtered.loc[start_date:end_date] # 시간 shorting
alog_filtered = alog_filtered[alog_filtered['how_0'] != '자동제어변경'] # 자동제어변경 제외

ahu_plus = alog_filtered[alog_filtered['type']=='ahu']
ahu_plus = ahu_plus.rename(columns={'dnumber':'ahu_no'})

chiller_plus = alog_filtered[alog_filtered['type']=='freezer']


ahu_plus = ahu_plus[ahu_plus['ahu_no']<=14] # A 빌딩안에 있는 1~14만 남김

start_date = '2023-01-31 10:00:00'
end_date = '2023-08-01 16:00:00'
ahu_group = ahu_plus.groupby('ahu_no')
ahu_plus_dct = {}

for name, group_df in ahu_group:
  ahu_plus_dct[name] = group_df
  # ahu_plus_dct[name].loc

<ipython-input-6-2b37b13d884c>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  alog['text'] = alog['text'].str.replace('::',',').str.replace('[','').str.replace(']',',').str.replace('\\','').str.replace(':',',')
<ipython-input-6-2b37b13d884c>:5: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  alog[['device', 'how_0', 'how_1', 'power']] = pd.DataFrame(alog.text.str.rsplit(',', 3).tolist())
<ipython-input-6-2b37b13d884c>:6: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  alog[['name', 'day']] = pd.DataFrame(alog.device.str.rsplit(',', 1).tolist())
<ipython-input-6-2b37b13d884c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [7]:
ahu_plus_dct[1].loc[:'2023-02-01 11'].head()

,type,mode,how_0,how_1,power,day,dname,ahu_no
cdate,,,,,,,,
2023-01-31 13:40:00,ahu,0,수동제어,전원,0,None,공조기,1
2023-01-31 19:00:00,ahu,1,스케줄제어,전원,0,None,공조기,1
2023-02-01 08:50:00,ahu,1,스케줄제어,전원,1,None,공조기,1
2023-02-01 11:40:00,ahu,0,수동제어,인버터,32Hz,None,공조기,1


# AHU 데이터 병합

In [8]:
ahu_fin_dict = {}

for i in range(1,15): # 총 ahu는 14개
    ### 스케줄 제어
    ahu_plus1 = ahu_plus_dct[i]
    ahu1 = ahu_dct[i]

    ahu_sch1 = ahu_plus1[ahu_plus1['how_0']=='스케줄제어']
    date_range = pd.date_range(start=start_date, end=end_date, freq='10T')
    ahu_sch1 = ahu_sch1.reindex(date_range) # 스케줄 제어를 10분 단위로 만든다. 결측치는 NaN으로
    ahu_sch1 = ahu_sch1.fillna(method='ffill') # ffill로 결측치를 채운다.
    ahu1_com = ahu1.combine_first(ahu_sch1)

    ### 수동제어
    ahu_man1 = ahu_plus1[ahu_plus1['how_0']=='수동제어']
    # ahu_onoff1 = ahu_man1[ahu_man1['how_1']=='전원'] # power는 0,1 로 이루어져 있다. 근데 기존데이터와 너무 겹치는것도 많고 어떤 데이터가 옿은지 모르겠음
    ahu_inv1 = ahu_man1[ahu_man1['how_1']=='인버터'] # power는 Hz 값으로 이루어져 있다. >> inv 칼럼으로 이동

    ahu_inv1 = ahu_man1[ahu_man1['how_1']=='인버터']
    ahu_inv1['method'] = ahu_inv1['how_1']

    date_range = pd.date_range(start=start_date, end=end_date, freq='10T')
    ahu_inv1 = ahu_inv1[~ahu_inv1.index.duplicated(keep='first')]
    ahu_inv1 = ahu_inv1.reindex(date_range) # 스케줄 제어를 10분 단위로 만든다. 결측치는 NaN으로
    ahu_inv1 = ahu_inv1.fillna(method='ffill') # ffill로 결측치를 채운다.
    ahu_inv1['power'] = ahu_inv1['power'].str.replace('Hz', '') # Hz 날린다.

    ahu_inv1.rename(columns={'power':'inv'}, inplace=True)
    ahu1_comm = ahu1_com.combine_first(ahu_inv1)
    # 인버터 수동제어하는 파트(method='인버터')만 how_0, how_1 각각 바꿔준다.
    ahu1_comm.loc[ahu1_comm['method'].notnull(), 'how_0'] = '수동제어'
    ahu1_comm.loc[ahu1_comm['method'].notnull(), 'how_1'] = '인버터'
    ahu1_comm.drop('method', axis=1, inplace=True)
    ahu_fin_dict[i] = ahu1_comm

<ipython-input-8-edc1ffcd1d47>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ahu_inv1['method'] = ahu_inv1['how_1']
<ipython-input-8-edc1ffcd1d47>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ahu_inv1['method'] = ahu_inv1['how_1']
<ipython-input-8-edc1ffcd1d47>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [12]:
ahu_fin_dict[1].iloc[10000:10020]

,ahu_no,day,dname,how_0,how_1,in_temp,inv,mode,occupancy,out_temp,power,type,zone_min_temp
2023-04-10 20:40:00,1.0,월,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,0,ahu,NaN
2023-04-10 20:50:00,1.0,월,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,0,ahu,NaN
2023-04-10 21:00:00,1.0,월,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,0,ahu,NaN
2023-04-10 21:10:00,1.0,월,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,0,ahu,NaN
2023-04-10 21:20:00,1.0,월,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,0,ahu,NaN
2023-04-10 21:30:00,1.0,월,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,0,ahu,NaN
2023-04-10 21:40:00,1.0,월,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,0,ahu,NaN
2023-04-10 21:50:00,1.0,월,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,0,ahu,NaN
2023-04-10 22:00:00,1.0,월,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,0,ahu,NaN
2023-04-10 22:10:00,1.0,월,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,0,ahu,NaN


In [10]:
ahu_fin_dict[14]

,ahu_no,day,dname,how_0,how_1,in_temp,inv,mode,occupancy,out_temp,power,type,zone_min_temp
2023-01-31 10:00:00,14.0,NaN,NaN,기존제어,NaN,\N,0.0,NaN,6.0,24.0,0.0,NaN,\N
2023-01-31 10:10:00,14.0,NaN,NaN,기존제어,NaN,\N,0.0,NaN,6.0,24.0,0.0,NaN,\N
2023-01-31 10:20:00,14.0,NaN,공조기,수동제어,인버터,\N,0.0,0.0,6.0,25.0,0.0,ahu,\N
2023-01-31 10:30:00,14.0,NaN,공조기,수동제어,인버터,\N,0.0,0.0,6.0,25.0,0.0,ahu,\N
2023-01-31 10:40:00,14.0,NaN,공조기,수동제어,인버터,\N,0.0,0.0,6.0,25.0,0.0,ahu,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01 15:20:00,14.0,화,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,1,ahu,NaN
2023-08-01 15:30:00,14.0,화,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,1,ahu,NaN
2023-08-01 15:40:00,14.0,화,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,1,ahu,NaN
2023-08-01 15:50:00,14.0,화,공조기,수동제어,인버터,NaN,45,1.0,NaN,NaN,1,ahu,NaN


In [11]:
# with open("/content/drive/MyDrive/ime_project/done/ahu_complete.pkl", 'wb') as file:
#     pickle.dump(ahu_fin_dict, file)